This program works 9 April 2025

In [ ]:
import matplotlib.pyplot as plt
from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.analysis.phase_diagram import PDPlotter

# Make sure plots appear in the notebook
%matplotlib inline
# For higher resolution plots
%config InlineBackend.figure_format = 'retina'

# Replace with your Materials Project API key
API_KEY = "Your API Key Goes Here" 

# Define the binary system to analyze
element1 = "Li"
element2 = "O"
chemsys = [element1, element2]

# Connect to the Materials Project database
with MPRester(API_KEY) as mpr:
    # Get all entries in the binary system
    entries = mpr.get_entries_in_chemsys(chemsys)
    
    # Create a phase diagram
    phase_diagram = PhaseDiagram(entries)
    
    # We'll create a new, customized plot entirely from scratch
    # This way we have complete control over the appearance
    
    # Create a new figure
    plt.figure(figsize=(12, 8))
    
    # Plot all entries
    for entry in entries:
        # Get composition and energy
        comp = entry.composition
        # Calculate fraction of second element
        if comp.is_element:
            if comp.elements[0] == phase_diagram.elements[1]:
                x = 1.0
            else:
                x = 0.0
        else:
            x = comp.get_atomic_fraction(phase_diagram.elements[1])
        
        # Get formation energy
        if comp.is_element:
            y = 0
        else:
            y = phase_diagram.get_form_energy_per_atom(entry)
        
        # Plot stable entries as red squares, unstable as smaller blue circles
        if entry in phase_diagram.stable_entries:
            plt.plot(x, y, 'rs', markersize=8)
            # Add text label slightly offset from the point
            plt.text(x, y+0.05, entry.name, fontsize=12, ha='center')
        else:
            e_above_hull = phase_diagram.get_e_above_hull(entry)
            if e_above_hull < 0.2:  # Only plot reasonably close metastable phases
                plt.plot(x, y, 'bo', markersize=6, alpha=0.7)
    
    # Plot the convex hull by connecting stable entries
    # First, collect all stable points
    hull_xs = []
    hull_ys = []
    
    for entry in phase_diagram.stable_entries:
        # Get composition
        comp = entry.composition
        # Calculate fraction of second element
        if comp.is_element:
            if comp.elements[0] == phase_diagram.elements[1]:
                x = 1.0
            else:
                x = 0.0
        else:
            x = comp.get_atomic_fraction(phase_diagram.elements[1])
        
        # Get formation energy
        if comp.is_element:
            y = 0
        else:
            y = phase_diagram.get_form_energy_per_atom(entry)
        
        hull_xs.append(x)
        hull_ys.append(y)
    
    # Sort the hull points by x-coordinate
    hull_points = sorted(zip(hull_xs, hull_ys))
    hull_xs = [point[0] for point in hull_points]
    hull_ys = [point[1] for point in hull_points]
    
    # Draw the convex hull
    plt.plot(hull_xs, hull_ys, 'k-', linewidth=2)
    
    # Add title and labels
    plt.title(f"{element1}-{element2} Binary Phase Diagram", fontsize=16)
    plt.xlabel(f"Fraction of {element2}", fontsize=14)
    plt.ylabel("Formation energy (eV/atom)", fontsize=14)
    
    plt.tight_layout()
    
    # Print information about the stable phases
    stable_entries = phase_diagram.stable_entries
    print(f"Number of stable phases: {len(stable_entries)}")
    print("Stable phases:")
    for entry in stable_entries:
        print(f"  {entry.name}")
    
    # Calculate and print formation energies
    print("\nStable compositions with formation energies:")
    for entry in stable_entries:
        if not entry.composition.is_element:  # Skip the pure elements
            form_e = phase_diagram.get_form_energy_per_atom(entry)
            print(f"  {entry.name}: {form_e:.3f} eV/atom")

In [ ]:
import matplotlib.pyplot as plt
from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.analysis.phase_diagram import PDPlotter

# Replace with your Materials Project API key
API_KEY = "Your API Key Goes Here" 

# Define the chemical system - a ternary system with Li, Fe, and O
chemsys = ["Li", "Fe", "O"]

# Connect to the Materials Project database and fetch entries
with MPRester(API_KEY) as mpr:
    # Get all entries in the system (both stable and unstable)
    entries = mpr.get_entries_in_chemsys(chemsys)
    
    # Create a phase diagram from these entries
    # This calculates the convex hull in composition space
    phase_diagram = PhaseDiagram(entries)
    
    # Create a plotter with the desired settings
    # - backend="plotly": Use plotly for interactive visualization
    # - show_unstable=0.2: Show unstable phases within 0.2 eV/atom above hull
    # - ternary_style="3d": Show the phase diagram in 3D for better visualization
    plotter = PDPlotter(phase_diagram, show_unstable=0.2, 
                       backend="plotly", ternary_style="3d")
    
    # Get the plot with stable phases labeled and hull facets filled
    plt_fig = plotter.get_plot(label_stable=True, fill=True)
    
    # Show the interactive plot
    plt_fig.show()
    
    # Print information about the stable phases
    stable_entries = phase_diagram.stable_entries
    print(f"Number of stable phases: {len(stable_entries)}")
    print("Stable phases:")
    for entry in stable_entries:
        print(f"  {entry.name}")